## Trabalho Final

In [1]:
import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer

from unicodedata import normalize

nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def trata_caracteres(txt):
    txt=[normalize('NFKD',x).encode('ASCII','ignore').decode('ASCII') for x in txt]
    txt=[re.sub('[^a-z|\ ]','',str.lower(x)) for x in txt]
    return txt

In [3]:
stop_words = set(stopwords.words('english'))
def remove_stop_words(txt):
    texto = CountVectorizer(ngram_range=(1,1),stop_words=stop_words)
    texto.fit(txt)
    return texto

In [4]:
def tratar_texto(txt,metodo=''):
    txt = trata_caracteres(txt)
    try:
        texto = remove_stop_words(txt)
        lista_txt = texto.get_feature_names()
    except:
        lista_txt = txt
    if metodo=='lemma':
        lem = WordNetLemmatizer()
        for part_of_speech in ['a', 's', 'r', 'n', 'v']:
            tms = [lem.lemmatize(a,part_of_speech) for a in lista_txt]
    if metodo == 'stem':
        ps = PorterStemmer()
        tms = [ps.stem(a) for a in lista_txt]
    if metodo == '':
        tms = txt
    return " ".join(tms)

In [5]:
def processamento_text(df):
    return df.apply(lambda x: tratar_texto([x],'stem'))

In [6]:
def compute_DTM(df):
    vect = CountVectorizer()
    H = vect.fit(df)
    o = vect.transform(df)
    return pd.DataFrame(o.A, columns=vect.get_feature_names())

In [7]:
def compute_DTM_ajustado(df,queries):
    vect = CountVectorizer()
    H = vect.fit(df)
    o = vect.transform(queries)
    return pd.DataFrame(o.A, columns=vect.get_feature_names())

In [8]:
def remove_ano(txt):
    txt=re.sub(r'(\([^a-z]*\))','',str(txt))
    return txt

In [9]:
data = pd.read_csv('movies.csv',index_col=0)

In [10]:
query = ['toy story', 'the lion king','alladin','beauty and the best','cinderella','little mermaid','hercules']

In [11]:
data['Ntitle'] = processamento_text(data.title)

In [12]:
X = compute_DTM(data.Ntitle)

In [13]:
N = len(query)

In [14]:
xij = compute_DTM_ajustado(data.Ntitle,processamento_text(pd.DataFrame(query)[0]))

In [15]:
m = X.mean(axis=0)
c = 2
B = c*(1-m)
a = c*m

In [16]:
a_til = a + xij.sum(axis=0)

In [17]:
B_til = B + N - xij.sum(axis=0)

In [18]:
q = np.log(a_til) - np.log(a) - np.log(B_til) + np.log(B)

In [19]:
nc = np.log(a + B) - np.log(a + B + N) + np.log(B_til) - np.log(B)

In [20]:
s = nc + X.multiply(q)

In [21]:
ndata = pd.DataFrame(data.title.apply(remove_ano)).reset_index()
score = pd.DataFrame(s.sum(axis=1),columns=['score'])      

In [22]:
result = ndata.join(score)    

In [23]:
result.sort_values(by='score',ascending=False)

,movieId,title,score
1997,2081,"Little Mermaid, The",9.898317
360,364,"Lion King, The",9.141145
9398,27619,"Lion King 1½, The",9.141145
7960,8643,"Cinderella Story, A",8.860324
0,1,Toy Story,8.503923
3027,3114,Toy Story 2,8.503923
15401,78499,Toy Story 3,8.503923
18646,92793,Rusalochka (The Little Mermaid),8.394211
2041,2125,Ever After: A Cinderella Story,7.355762
18252,91266,Another Cinderella Story,7.355276
